In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("/content/Emotion_final.csv")

In [ ]:
df

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
21454,Melissa stared at her friend in dism,fear
21455,Successive state elections have seen the gover...,fear
21456,Vincent was irritated but not dismay,fear
21457,Kendall-Hume turned back to face the dismayed ...,fear


In [ ]:
df['Emotion'].value_counts()

happy       7029
sadness     6265
anger       2993
fear        2652
love        1641
surprise     879
Name: Emotion, dtype: int64

In [ ]:
df['Emotion'].replace({"happy":0,"sadness":1,"anger":2,"fear":3,"love":4,"surprise":5},inplace=True)

In [ ]:
df

,Text,Emotion
0,i didnt feel humiliated,1
1,i can go from feeling so hopeless to so damned...,1
2,im grabbing a minute to post i feel greedy wrong,2
3,i am ever feeling nostalgic about the fireplac...,4
4,i am feeling grouchy,2
...,...,...
21454,Melissa stared at her friend in dism,3
21455,Successive state elections have seen the gover...,3
21456,Vincent was irritated but not dismay,3
21457,Kendall-Hume turned back to face the dismayed ...,3


In [ ]:
df.isnull().sum()

Text       0
Emotion    0
dtype: int64

In [ ]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [ ]:
df["clean_text"]=df["Text"].apply(cleantext)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["Emotion"] = le.fit_transform(df["Emotion"])

In [ ]:
df

,Text,Emotion,clean_text
0,i didnt feel humiliated,1,didnt feel humiliated
1,i can go from feeling so hopeless to so damned...,1,go feeling hopeless damned hopeful around some...
2,im grabbing a minute to post i feel greedy wrong,2,im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...,4,ever feeling nostalgic fireplace know still pr...
4,i am feeling grouchy,2,feeling grouchy
...,...,...,...
21454,Melissa stared at her friend in dism,3,melissa stared friend dism
21455,Successive state elections have seen the gover...,3,successive state election seen governing party...
21456,Vincent was irritated but not dismay,3,vincent irritated dismay
21457,Kendall-Hume turned back to face the dismayed ...,3,turned back face dismayed coup


In [ ]:
x = df["clean_text"]
y = df["Emotion"]

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
sentlen = []

for sent in df["clean_text"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen
df.head()

,Text,Emotion,clean_text,SentLen
0,i didnt feel humiliated,1,didnt feel humiliated,3
1,i can go from feeling so hopeless to so damned...,1,go feeling hopeless damned hopeful around some...,9
2,im grabbing a minute to post i feel greedy wrong,2,im grabbing minute post feel greedy wrong,7
3,i am ever feeling nostalgic about the fireplac...,4,ever feeling nostalgic fireplace know still pr...,7
4,i am feeling grouchy,2,feeling grouchy,2


In [ ]:
max(sentlen)

35

In [ ]:
np.quantile(sentlen,0.95)

19.0

In [ ]:
max_len = np.quantile(sentlen, 0.95)

In [ ]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'feel',
 2: 'feeling',
 3: 'like',
 4: 'im',
 5: 'time',
 6: 'really',
 7: 'know',
 8: 'get',
 9: 'little',
 10: 'make',
 11: 'day',
 12: 'people',
 13: 'want',
 14: 'thing',
 15: 'one',
 16: 'would',
 17: 'life',
 18: 'think',
 19: 'even',
 20: 'still',
 21: 'ive',
 22: 'much',
 23: 'bit',
 24: 'going',
 25: 'way',
 26: 'love',
 27: 'something',
 28: 'could',
 29: 'dont',
 30: 'go',
 31: 'back',
 32: 'need',
 33: 'good',
 34: 'pretty',
 35: 'see',
 36: 'always',
 37: 'right',
 38: 'say',
 39: 'work',
 40: 'also',
 41: 'year',
 42: 'today',
 43: 'made',
 44: 'around',
 45: 'friend',
 46: 'felt',
 47: 'though',
 48: 'help',
 49: 'cant',
 50: 'got',
 51: 'thought',
 52: 'look',
 53: 'well',
 54: 'happy',
 55: 'never',
 56: 'someone',
 57: 'le',
 58: 'didnt',
 59: 'lot',
 60: 'come',
 61: 'every',
 62: 'take',
 63: 'week',
 64: 'quite',
 65: 'find',
 66: 'away',
 67: 'kind',
 68: 'many',
 69: 'anything',
 70: 'last',
 71: 'left',
 72: 'said',
 73: 'new',
 74: 'sure',
 75: 'home',
 76:

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

14146

In [ ]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[71, 6802, 1373, 24, 1, 242, 2503, 153, 1099, 1374, 3195, 24, 744],
 [1, 26, 1620, 651, 221, 714, 12, 4, 292, 142, 73, 299, 17, 6803],
 [13, 1, 715],
 [16, 1, 6, 530],
 [152, 2, 72, 617, 3196],
 [4784, 6804, 1, 1621, 258, 1445],
 [90, 2, 34, 33],
 [1, 4, 82, 57, 57, 2252],
 [2, 1525, 24, 336, 634, 562, 411, 116, 3795, 1823, 745, 88, 3796],
 [1, 273, 805, 59, 1824, 18, 8, 965],
 [4, 2, 247, 42, 3197, 15, 149, 116, 763, 33, 6805],
 [353, 1, 14, 425, 67, 364, 58, 1, 1526],
 [1, 581, 40, 19, 716],
 [1,
  3,
  6806,
  197,
  6807,
  259,
  15,
  6808,
  788,
  31,
  27,
  46,
  189,
  41,
  337,
  2504,
  1527,
  162,
  426,
  2080,
  2505,
  17],
 [4, 134, 879, 425, 4, 65, 936, 563, 10, 1, 427, 53, 835],
 [30, 255, 69, 2, 465, 378, 1719, 3797, 1375],
 [6, 1, 439, 205, 2253, 4785, 4786, 2253, 6809, 4786, 605, 1058],
 [1, 1376, 17, 764, 201],
 [1, 338, 272, 219],
 [7, 746, 1, 477, 1100, 698, 24, 510],
 [65, 103, 6810, 1059, 1038, 1, 211, 765],
 [29, 7, 4787, 1, 494, 679],
 [125, 6811, 5, 37

In [ ]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[    0,     0,     0, ...,  3195,    24,   744],
       [    0,     0,     0, ...,   299,    17,  6803],
       [    0,     0,     0, ...,    13,     1,   715],
       ...,
       [    0,     0,     0, ...,     8,   161,    27],
       [    0,     0,     0, ...,   288, 14146,   380],
       [    0,     0,     0, ...,  1808,    15,     5]], dtype=int32)

In [ ]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [ ]:
seqmattest

array([[    0,     0,     0, ...,     2,   219,   178],
       [    0,     0,     0, ...,     6,   527, 10290],
       [    0,     0,     0, ...,    15,   341,   526],
       ...,
       [    0,     0,     0, ...,   201,   122,   578],
       [  118,   345,  2065, ...,    16,   322,   529],
       [    0,     0,     0, ...,     2,   179,    42]], dtype=int32)

In [ ]:
vocab_len

14146

In [ ]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,15, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=6, activation="softmax"))


rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 15)            212205    
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1536      
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
Total params: 214,995
Trainable params: 214,995
Non-trainable params: 0
_________________________________________________________________


In [ ]:
rnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy",metrics=['accuracy'])

In [ ]:

history=rnn.fit(seqmattrain, ytrain, validation_split=0.2, epochs=10)


Epoch 1/10
376/376 [==============================] - 41s 89ms/step - loss: 1.5090 - accuracy: 0.3585 - val_loss: 1.2083 - val_accuracy: 0.4819
Epoch 2/10
376/376 [==============================] - 18s 48ms/step - loss: 0.8684 - accuracy: 0.6738 - val_loss: 0.8151 - val_accuracy: 0.7205
Epoch 3/10
376/376 [==============================] - 17s 46ms/step - loss: 0.4422 - accuracy: 0.8460 - val_loss: 0.7202 - val_accuracy: 0.7671
Epoch 4/10
376/376 [==============================] - 16s 42ms/step - loss: 0.2792 - accuracy: 0.9075 - val_loss: 0.7366 - val_accuracy: 0.7704
Epoch 5/10
376/376 [==============================] - 14s 38ms/step - loss: 0.1882 - accuracy: 0.9412 - val_loss: 0.8154 - val_accuracy: 0.7784
Epoch 6/10
376/376 [==============================] - 14s 38ms/step - loss: 0.1210 - accuracy: 0.9653 - val_loss: 0.8929 - val_accuracy: 0.7824
Epoch 7/10
376/376 [==============================] - 16s 42ms/step - loss: 0.0874 - accuracy: 0.9748 - val_loss: 1.0228 - val_accuracy:

In [ ]:
seqmattest[0]

array([   0,    0,    0,   29,    7, 1767, 1461, 2189,  418,   56,   26,
       2207,   42,   18, 4956,   14,    2,  219,  178], dtype=int32)

In [ ]:
yprob=rnn.predict(seqmattest)
yprob[0]

202/202 [==============================] - 1s 5ms/step


array([5.4186861e-10, 2.4573495e-05, 8.1013435e-01, 1.8984067e-01,
       4.4265670e-07, 5.9808308e-08], dtype=float32)

In [ ]:
ypred=yprob.argmax(axis=1)
ypred

array([2, 5, 3, ..., 4, 4, 2])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      2123
           1       0.79      0.86      0.83      1922
           2       0.72      0.69      0.71       897
           3       0.66      0.69      0.67       769
           4       0.56      0.54      0.55       477
           5       0.61      0.48      0.54       250

    accuracy                           0.77      6438
   macro avg       0.70      0.68      0.69      6438
weighted avg       0.77      0.77      0.77      6438

